<a href="https://colab.research.google.com/github/DhruvJ2k4/Query_Anomaly_detector/blob/main/PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
import pandas as pd
from datasets import load_dataset, concatenate_datasets
import torch
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
from google.colab import drive, files

In [ ]:
train2 = pd.read_parquet("hf://datasets/togoron/search-query-json/data/train-00000-of-00001.parquet")

In [ ]:
dataset1 = load_dataset("EmbeddingStudio/synthetic-search-queries-ru")

In [ ]:
dataset1 = concatenate_datasets([dataset1['train'], dataset1['test']])
dataset1 = dataset1.shuffle(seed=42)

In [ ]:
train1 = pd.DataFrame(dataset1)
del train1['Parsed']
del train1['category']

In [ ]:
train1.head()

,Query
0,"Встречусь с человеком, который живет в Лос-Анд..."
1,"Нужен скрипт для CI, который будет запускаться..."
2,Анализ рисков за период с 15.07.2021 по 31.12....
3,Я ищу информацию о социальных сетях с координа...
4,оформить страховку без указания пола


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load translation model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-ru-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
#Translate function
def translate(texts, batch_size=8):
    translations = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            translated = model.generate(**inputs)
        outputs = tokenizer.batch_decode(translated, skip_special_tokens=True)
        translations.extend(outputs)
    return translations

In [ ]:
#Translate the column
texts = train1["Query"].astype(str).tolist()
train1[f"Query_en"] = translate(texts)

100%|██████████| 1404/1404 [13:37<00:00,  1.72it/s]


In [ ]:
train1.head()

,Query_en
0,Meet a man who lives in L.A. and he's 23 years...
1,"Need a script for the CI, which will be launch..."
2,Risk analysis from 15.07.2021 to 31.12.2021
3,I'm looking for information on social media at...
4,:: Provide insurance without a gender indicati...


In [ ]:
del train2['assistant']
train2.head()

,user
0,How can I bake a cookie?
1,What is the world's highest building?
2,Tell me a joke.
3,How to reset my phone?
4,What's the weather like in Paris today?


In [ ]:
queryData = pd.concat([train1['Query_en'], train2['user']], ignore_index=True)
queryData = queryData.to_frame(name="Query")

In [ ]:
queryData.head()

,Query
0,Meet a man who lives in L.A. and he's 23 years...
1,"Need a script for the CI, which will be launch..."
2,Risk analysis from 15.07.2021 to 31.12.2021
3,I'm looking for information on social media at...
4,:: Provide insurance without a gender indicati...


In [ ]:
#Save the DataFrame to a CSV file
queryData.to_csv("query_data.csv", index=False)
files.download("query_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>